In [1]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import src.functions as src

fs=500
freq_bands={'delta' : [0,4],
            'theta' : [4,8],
            'alpha' : [8, 13],
            'beta' : [13,35],
            'high' : [35, 45],
            'all': [1.5,45],
            'low': [1.5,12]}

In [ ]:
data1 = {}
labels1 = {}
file1= [item for item in os.listdir('data') if item.endswith('set')]

for file_index, file_name in enumerate(file1):
    path_ = os.path.join( 'data', file_name )
    dados = mne.read_epochs_eeglab(path_)
    channel_names = dados.info['ch_names']
    subject_id = file_name[:3]
    
    # Extract labels
    proCorr_antiCorr_labels = len(dados['proCorr','antiCorr'].get_data())
    proErr_antiErr_nogoErr_labels = len(dados['proErr','antiErr','nogoErr'].get_data())
    labels1[subject_id] = np.array([0] * proCorr_antiCorr_labels + [1] * proErr_antiErr_nogoErr_labels)
    
    # Initialize channel data dictionary
    channel_data = {}
    
    # Iterate over 'proCorr' and 'antiCorr' events
    for epoch_values in dados['proCorr','antiCorr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    # Iterate over 'proErr', 'antiErr', and 'nogoErr' events
    for epoch_values in dados['proErr','antiErr','nogoErr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    data1[subject_id] = channel_data


In [ ]:
data2={}
labels2={}
path='Dataset2'
for files in os.listdir(path):
    folders=os.path.join(path,files)
    file2=[item for item in os.listdir(folders) if item.endswith('set')]
#--------------------------------------------------------------------------
    i=0
    label=[]
    while i!=4:
        for file_index, file_name in enumerate(file2):
            i=i+1
            path_ = os.path.join( folders, file_name )
            dados = mne.read_epochs_eeglab(path_)
            channel_names = dados.info['ch_names']
            subject_id = file_name[:3]
            
            # Extract labels
            try:
                Corr_labels = int(len(dados['150','160'].get_data()))
            except:
                Corr_labels=0
            try:
                Err_labels = int(len(dados['151','152','153','154','161','162','163','164'].get_data()))
            except:
                Err_labels=0
            label.append(np.array([0] * Corr_labels + [1] * Err_labels))
           
    #--------------------------------------------------------------------------    
            # Initialize channel data dictionary
            channel_data = {}
            data_channel=[]
            try:
                for epoch_values in dados['150','160'].get_data():
                    for channel_index, channel_values in enumerate(epoch_values):
                        channel_name = channel_names[channel_index]
                        if channel_name not in channel_data:
                            channel_data[channel_name] = []
                        channel_data[channel_name].append(list(channel_values))
            except:
                pass
            try:
                for epoch_values in dados['151','152','153','154','161','162','163','164'].get_data():
                    for channel_index, channel_values in enumerate(epoch_values):
                        channel_name = channel_names[channel_index]
                        if channel_name not in channel_data:
                            channel_data[channel_name] = []
                        channel_data[channel_name].append(list(channel_values))
            except:
                pass
            
            data2[file_name] = channel_data
        labels2[subject_id] = np.concatenate(label)

#--------------------------------------------------------------------------

rearranged_data = {}

# Iterate through each file in data2
for file_name, channel_data in data2.items():
    # Extract the subject ID from the file name
    subject_id = file_name.split('_')[0]
    
    # Initialize the subject in the rearranged_data dictionary if not already present
    if subject_id not in rearranged_data:
        rearranged_data[subject_id] = {}
    
    # Iterate through each channel in the current file
    for channel, data in channel_data.items():
        # Initialize the channel in the subject's dictionary if not already present
        if channel not in rearranged_data[subject_id]:
            rearranged_data[subject_id][channel] = []
        
        # Append the current file's data for the channel to the subject's channel data
        rearranged_data[subject_id][channel].append(data)

# Combine the data from different files of the same subject for each channel
for subject_id, channels in rearranged_data.items():
    for channel, data_list in channels.items():
        # Flatten the list of data arrays into a single list or concatenate them
        # Here we assume each data is a list or array; if not, adjust accordingly
        combined_data = []
        for data in data_list:
            combined_data.extend(data)
        rearranged_data[subject_id][channel] = combined_data

# Now rearranged_data is in the desired format
# You can print or further process the rearranged_data as needed

data2=rearranged_data
        

In [ ]:
def data_pre_post(data):
    pre_response={}
    post_response={}
    for subject_id, dados in data.items():
        channel_data_pre={}
        channel_data_post={}
        for channel_name, values in dados.items():
            values_pre=[]
            values_post=[]
            for i in range(len(values)):
                
                index= int(len(values[i])/2)
                values_pre.append(values[i][0:index])
                values_post.append(values[i][index:-1])
            channel_data_pre[channel_name]= values_pre
            channel_data_post[channel_name]= values_post
            
        pre_response[subject_id] = channel_data_pre #(14,60,n_events,250)
        post_response[subject_id] = channel_data_post
    return pre_response, post_response


In [ ]:
pre_response1, post_response1 = data_pre_post(data1)

In [ ]:
pre_response2, post_response2 = data_pre_post(data2)